# Understanding the Core Objects in Deeplay

Before starting to implement applications in Deeplay, let's define the most important objects for building a model in Deeplay:

- **Application:** The highest-level object that defines the neural network architecture and training process. This includes the loss, the optimizer, and the training logic. Applications are typically task-oriented, such as `ImageClassifier` and `ObjectDetector`.

- **Model:** A model is a specific neural network architecture that is typically part of an application. Examples include `ResNet` and `VGG`.

- **Component:** A model is usually made by combining multiple components. Components are much more flexible than models. Examples include `MultiLayerPerceptron` and `ConvolutionalEncoder2d`.

- **Block:** A block is a specific combination of layers that performs a small unit of calculation (for example layer plus activation). Blocks are the building blocks of components. They are the most flexible objects in Deeplay. Examples include `LinearBlock` and `Conv2dBlock`.

- **Layer:** A layer consists of a single torch layer, such as `torch.nn.Linear` and `torch.nn.Conv2d`. Layers are the most basic building blocks in Deeplay.

In the following sections, you'll create some examples of these obejcts.

## Importing Deeplay

Import `deeplay` (shortened to `dl`, as an abbreaviation of both *deeplay* and *deep learning*) ... 

In [1]:
import deeplay as dl

... and the `torch.nn` namespace.

In [2]:
import torch.nn as nn

## Layers

Starting with the most basic building block, a `Layer` is a single PyTorch layer. In this example, you'll create a linear layer with 10 input features and 5 output features.

In [3]:
linear_layer = dl.Layer(nn.Linear, in_features=10, out_features=5)

print(linear_layer)

Layer[Linear](in_features=10, out_features=5)


You can modify the layer after it's created using the `configure()` method to change any of its properties. For example, here you'll change the number of output features and remove the bias.

In [4]:
linear_layer.configure(out_features=3, bias=False)

print(linear_layer)

Layer[Linear](in_features=10, out_features=3, bias=False)


To make the layer into a pure PyTorch module, you simply need to build it.

In [5]:
torch_layer = linear_layer.build()

print(torch_layer)

Linear(in_features=10, out_features=3, bias=False)


**NOTE:** Most Deeplay objects are modified in place when you build them. If you want to keep the original object, either call `.create()` or `.new().build()` instead; in this way, layers will return torch objects, while most other deeplay objects will return Deeplay objects.

## Blocks

Going up one level in the hierarchy, a `Block` is a combination of layers that performs a relatively simple calculation. They are usually a sequence of `Layer` objects, but can include other blocks or sequences of blocks as well. In this example, you'll create a block that consists of a linear layer followed by a ReLU activation function.

Start by creating a `LinearBlock`...

In [6]:
linear_block = dl.LinearBlock(in_features=10, out_features=5)

print(linear_block)

LinearBlock(
  (layer): Layer[Linear](in_features=10, out_features=5, bias=True)
)


While there is no activation by default, there are a few ways to add an activation to a block. 

The most common way to add an activation is to use the `activation` argument when creating the block ...

In [7]:
linear_block_with_activation = dl.LinearBlock(
    in_features=10, 
    out_features=5, 
    activation=dl.Layer(nn.ReLU),
).build()

print(linear_block_with_activation)

LinearBlock(
  (layer): Linear(in_features=10, out_features=5, bias=True)
  (activation): ReLU()
)


... you can also add an activation to an existing block using the `.activated()` method ...

In [8]:
linear_block_activated = dl.LinearBlock(in_features=10, out_features=5)

linear_block_activated.activated(nn.ReLU)

print(linear_block_activated)

LinearBlock(
  (layer): Layer[Linear](in_features=10, out_features=5, bias=True)
  (activation): Layer[ReLU]()
)


... you can use the `.configure()` method to add an activation to a block (this way is rarely needed) ...

In [9]:
linear_block_configured = dl.LinearBlock(in_features=10, out_features=5)

linear_block_configured.configure(activation=dl.Layer(nn.ReLU))

print(linear_block_configured)

LinearBlock(
  (layer): Layer[Linear](in_features=10, out_features=5, bias=True)
  (activation): Layer[ReLU]()
)


... or you can use the `.append()` method (also this way is rarely used).

In [10]:
linear_block_appended = dl.LinearBlock(in_features=10, out_features=5)

linear_block_appended.append(dl.Layer(nn.ReLU), name="activation")

print(linear_block_appended)

LinearBlock(
  (layer): Layer[Linear](in_features=10, out_features=5, bias=True)
  (activation): Layer[ReLU]()
)


## Components

A `Component` is a collection of blocks combined to form a more complex neural network component. 

In this example, you'll create a simple feedforward neural network component with two linear blocks, each followed by a ReLU activation function.

In [11]:
mlp_component = dl.MultiLayerPerceptron(
    in_features=10,
    hidden_features=[32],
    out_features=5,
    out_activation=dl.Layer(nn.ReLU)
)   

print(mlp_component)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=32, bias=True)
      (activation): Layer[ReLU]()
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=32, out_features=5, bias=True)
      (activation): Layer[ReLU]()
    )
  )
)


Since the component is made of blocks, you can access the blocks using the `.blocks` attribute. This allows you to modify the blocks after the component has been created. For example, you can change the activation function of the first block to a Sigmoid function ...

In [12]:
mlp_component.blocks[0].activated(nn.Sigmoid)

print(mlp_component)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=32, bias=True)
      (activation): Layer[Sigmoid]()
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=32, out_features=5, bias=True)
      (activation): Layer[ReLU]()
    )
  )
)


... or you can remove the activation.

In [13]:
mlp_component.blocks[0].remove("activation")

print(mlp_component)

MultiLayerPerceptron(
  (blocks): LayerList(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=32, bias=True)
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=32, out_features=5, bias=True)
      (activation): Layer[ReLU]()
    )
  )
)


## Models

The next step up in the hierarchy is a `Model`, which is a specific neural network architecture. Examples are lecun-5 and resnet. For fully connected networks, there are few widely recognized standard models. 

Here, you'll simply instantiate a small MLP model.

In [14]:
small_mlp = dl.models.SmallMLP(in_features=10, out_features=5)

print(small_mlp)

SmallMLP(
  (blocks): LayerList(
    (0): LinearBlock(
      (layer): Layer[Linear](in_features=10, out_features=32, bias=True)
      (activation): Layer[LeakyReLU](negative_slope=0.05)
      (normalization): Layer[BatchNorm1d]()
    )
    (1): LinearBlock(
      (layer): Layer[Linear](in_features=32, out_features=32, bias=True)
      (activation): Layer[LeakyReLU](negative_slope=0.05)
      (normalization): Layer[BatchNorm1d]()
    )
    (2): LinearBlock(
      (layer): Layer[Linear](in_features=32, out_features=5, bias=True)
      (activation): Layer[Identity]()
    )
  )
)


## Applications

Finally, an `Application` is the highest level of abstraction in Deeplay. It defines the neural network architecture and training process, including the loss, optimizer, and training logic. 

In this example, you'll create a simple classifier application that uses the previously defined model.

In [15]:
classifier = dl.Classifier(small_mlp, optimizer=dl.Adam(lr=0.001))

print(classifier)

Classifier(
  (loss): CrossEntropyLoss()
  (optimizer): Adam[Adam](lr=0.001)
  (train_metrics): MetricCollection,
    prefix=train
  )
  (val_metrics): MetricCollection,
    prefix=val
  )
  (test_metrics): MetricCollection,
    prefix=test
  )
  (model): SmallMLP(
    (blocks): LayerList(
      (0): LinearBlock(
        (layer): Layer[Linear](in_features=10, out_features=32, bias=True)
        (activation): Layer[LeakyReLU](negative_slope=0.05)
        (normalization): Layer[BatchNorm1d]()
      )
      (1): LinearBlock(
        (layer): Layer[Linear](in_features=32, out_features=32, bias=True)
        (activation): Layer[LeakyReLU](negative_slope=0.05)
        (normalization): Layer[BatchNorm1d]()
      )
      (2): LinearBlock(
        (layer): Layer[Linear](in_features=32, out_features=5, bias=True)
        (activation): Layer[Identity]()
      )
    )
  )
)


**NOTE:** The Deeplay optimizer is a wrapper around the torch optimizer and is used to delay the attribution of parameters from the models until after the model is actually created.